# Web Scraping Certificado ABR Fardos de Algodão
#### <font color = blue> Função: Realizar consultas do certificado ABR no site da Abrapa fardo a fardo
Automatização do processo de coleta da numeração dos certificados

## Importando bibliotecas e definindo funções

In [1]:
# importando bibliotecas
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
# configurando o header do navegador Chrome
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

In [3]:
# definindo função de requisição do site
def ConsultaWeb(url):
    try:
        req = requests.get(url, headers = headers, timeout = 3) 
        return req.text
    
    except:
        pass
        print('Tentativa sem sucesso, verificar link')

In [4]:
# definindo função para retornar o html da página
def captura_html_pagina(url):
    html = ConsultaWeb(url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [5]:
# definindo função que coleta a informação da tag html onde encontra-se a informação do certificado ABR
def Certificado(html):
    cert = html.find('div', {'class' : 'col-md-6'}).find_all('span')
    y = cert[-2].get_text()
    return y

In [6]:
# definindo função que coleta a informação e popula um dicionário com o par Fardo x Certificado
def FardoCabecalho(fardo):
    url = 'https://www.abrapa.com.br/Paginas/Rastreabilidade/Rastreabilidade%20-%20Resultados.aspx?cb='+str(fardo)
    html = captura_html_pagina(url)
    dic = {}
    dic['Fardo'] = fardo
    dic['Certificado ABR'] = Certificado(html)
    return dic

In [13]:
# definindo função de loop de consultas com iterações de todos elementos da lista completa de fardos
def TabelaConsulta():
    
    consulta = []
    consulta_erro = []
    contador = 0
    erro = 0
    
    for fd in lista_relacao:
    
      try:
        consulta = consulta + [FardoCabecalho(fd)]
        contador += 1
        
        print(contador, 'consulta(s) efetuada(s)', '| Fardo:', fd)
        time.sleep(1.5) #acrescentando segundos de espaço entre cada interação
      
      except:
        contador += 1
        erro += 1
        consulta_erro = consulta_erro + [fd]
        print(contador, 'consulta(s) efetuada(s) | Fardo com erro:', fd)
        pass
    
    
    print('\n\nOperação concluída |', erro, 'Fardo(s) apresentaram erro(s) na consulta:')
    print('\n', consulta_erro)

    tabela = pd.DataFrame(consulta)
    
    return tabela

## Importando a relação de fardos

In [8]:
# Importando relação de fardos do arquivo excel
relacao = pd.read_excel('RelacaoFardos.xlsx', dtype = str)

In [9]:
# Convertendo coluna 'Fardo' do arquivo em lista:
lista_relacao = relacao['Fardo'].tolist()

## Executando script & exportando consulta

In [ ]:
ConsultaFardos = TabelaConsulta()

1 consulta(s) efetuada(s) | Fardo: 00078986082711464128
2 consulta(s) efetuada(s) | Fardo: 00078986082711464173


In [11]:
ConsultaFardos

,Fardo,Certificado ABR
0,00078986082711464128,357.031.MT.2022
1,00078986082711464173,357.031.MT.2022
2,00078986082711464272,357.031.MT.2022
3,00078986082711464388,357.031.MT.2022
4,00078986082711473205,357.031.MT.2022
5,00078986082711473229,357.031.MT.2022
6,00078986082711473236,357.031.MT.2022
7,00078985385516101542,MT


In [12]:
# Salvando arquivo em csv
ConsultaFardos.to_csv('ConsultaFardos.txt', sep=";", encoding='latin-1', index= None)
